Available as executable or viewable Jupyter Notebook:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/HIL-HK/lets-plot-examples/master?filepath=plans%2Fminard.ipynb)

[<img alt="nbviewer" src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.png" width="109" height="20">](https://nbviewer.jupyter.org/github/HIL-HK/lets-plot-examples/blob/master/plans/minard.ipynb)

In [1]:
import pandas as pd

from lets_plot import *
from lets_plot.plot import geom_livemap; LetsPlot.setup_html()

Let's use [Wikimedia Maps](https://foundation.wikimedia.org/wiki/Maps_Terms_of_Use) provided by Wikimedia Foundation as map tile server.

Internet connection is required.

In [2]:
LetsPlot.set(maptiles_zxy(url='https://maps.wikimedia.org/osm-intl/{z}/{x}/{y}@2x.png'))

# Exploring Minard's 1812 Plot with Lets-Plot

In [3]:
PLOT_TITLE = "Charles Minard's 1869 Chart"
PLOT_SIZE = (1000, 400)

FORWARD_PATH_COLOR = '#dfc17e'
BACKWARD_PATH_COLOR = '#252523'
MARKS_COLOR = '#bb0000'

In [4]:
cities = pd.read_csv('https://raw.githubusercontent.com/HIL-HK/lets-plot-examples/master/data/minard/cities.csv')
troops = pd.read_csv('https://raw.githubusercontent.com/HIL-HK/lets-plot-examples/master/data/minard/troops.csv')
temps = pd.read_csv('https://raw.githubusercontent.com/HIL-HK/lets-plot-examples/master/data/minard/temps.csv')

## Basic Version

In [5]:
ggplot(troops) + \
    geom_path(aes(x='long', y='lat', group='group', color='direction', size='survivors')) + \
    scale_size(range=(.5, 15)) + \
    scale_color_manual(values=[FORWARD_PATH_COLOR, BACKWARD_PATH_COLOR]) + \
    geom_point(aes(x='long', y='lat'), data=cities, color=MARKS_COLOR) + \
    geom_text(aes(x='long', y='lat', label='city'), data=cities, vjust='top', color=MARKS_COLOR) + \
    ggtitle(PLOT_TITLE) + \
    ggsize(*PLOT_SIZE) + \
    theme(legend_position='none', axis_title='blank', axis_text='blank', axis_ticks='blank', axis_line='blank')

## With Livemap

In [6]:
ggplot(troops) + \
    geom_livemap(location=[23.5, 53.4, 38.1, 56.3]) + \
    geom_path(aes(x='long', y='lat', group='group', color='direction', size='survivors')) + \
    scale_size(range=(.5, 15)) + \
    scale_color_manual(values=[FORWARD_PATH_COLOR, BACKWARD_PATH_COLOR]) + \
    geom_point(aes(x='long', y='lat'), data=cities, color='red') + \
    ggtitle(PLOT_TITLE) + \
    ggsize(*PLOT_SIZE) + \
    theme(legend_position='none', axis_title='blank', axis_text='blank', axis_ticks='blank', axis_line='blank')

## How It Is Expected to Work

In [7]:
plot = ggplot()

dfs = [troops[i:(i + 2)] for i in range(troops.shape[0] - 2)]
dfs = filter(lambda df: df['group'].iloc[0] == df['group'].iloc[1], dfs)
MAX_PATH_WIDTH = 15
for df in dfs:
    plot += geom_path(
        aes(x='long', y='lat', group='group'),
        data=df,
        size=MAX_PATH_WIDTH * df['survivors'].iloc[0] / troops['survivors'].max(),
        color=(FORWARD_PATH_COLOR if df['direction'].iloc[0] == 'A' else BACKWARD_PATH_COLOR)
    )

plot + \
    geom_point(aes(x='long', y='lat'), data=cities, color=MARKS_COLOR) + \
    geom_text(aes(x='long', y='lat', label='city'), data=cities, vjust='top', color=MARKS_COLOR) + \
    ggtitle(PLOT_TITLE) + \
    ggsize(*PLOT_SIZE) + \
    theme(legend_position='none', axis_title='blank', axis_text='blank', axis_ticks='blank', axis_line='blank')